# nbcleanse CLI Interface

In [8]:
#| default_exp cli

In [30]:
#| export

from pathlib import Path
from nbcleanse.notebook import Notebook


def nbcleanse(**kwargs):
    "Clean all notebooks in the current directory using the given parameters"    
    for path in Path('.').glob('*.ipynb'):
        nb = Notebook(path)
        for k in kwargs:
            getattr(nb, k)()
        nb.save()

In [29]:
#| export

from argparse import ArgumentParser

# using argparse rather than fastcore.script because it's possible to add single dash, concatable flags

parser = ArgumentParser(prog='nbcleanse')
parser.add_argument('-m', action='store_true', help='Preserve cells and notebook-wide metadata', dest='clean_all_metadata')
parser.add_argument('-o', action='store_true', help='Preserve cells outputs', dest='clean_outputs')

def main():
    args = parser.parse_args()
    nbcleanse(**vars(args))

## export

In [31]:
from nbdev.export import nb_export

nb_export('cli.ipynb')